## Coursera Capstone: Segmenting and Clustering Neighborhoods in Toronto
This project is to explore, segment, and cluster neighborhoods in the city of Toronto.

## Introduction

We will start by scraping the Wikipedia page <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">List of postal codes of Canada: M</a> to get a list of neighborhoods in Toronto. Then we will find the latitude and longitude of each neighborhood. Next, we will use the Foursquare API to explore the neighborhoods. We will get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. We will use the *k*-means clustering algorithm to complete this task. Finally, we will use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

## Table of Contents

1. <a href="#toc1">Acquire Data</a>
2. <a href="#toc2">Assign Geographical Coordinates</a>

**This is the second notebook in the analysis. Additional sections will be added in subsequent submissions.**

First download all the dependencies that we will need for this analysis.

In [1]:
import pandas as pd # library for data analysis

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


<a id="toc1"></a>
## 1. Acquire Data

We will scrape the list of Toronto neighborhoods from the Wikipedia page <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">List of postal codes of Canada: M</a>. Postal codes beginning with M are located within the city of Toronto. Then we will find the geographical coordinates of each post code.

#### Read data from web page

We will read the page and convert the table into a dataframe using the *pandas* read_html method.

In [2]:
#get Toronto neighborhoods from Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#read the Wikipedia page - returns list of dataframes
dfs = pd.read_html(url, header=0)
#take the first dataframe from the returned list (it should be the only dataframe in the list)
df = dfs[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


We will ignore any records where *Borough* is "Not assigned".

In [3]:
#create new dataframe with records where Borough is not 'Not assigned'
df_assigned = df[df['Borough'] != 'Not assigned']
df_assigned.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In rows where the value of *Neighbourhood* is 'Not assigned' we will replace it with the value of the *Borough*.

In [4]:
#create a list of neighborhoods, replacing the borough where neighborhood is 'Not assigned'
new_neigh = df_assigned['Neighbourhood'].where(df_assigned['Neighbourhood'] != 'Not assigned', other = df_assigned['Borough'], axis = 0)
#construct new dataframe using postcode and borough from the previous dataframe and neighborhood from the above list
df_replaced = pd.concat([df_assigned['Postcode'], df_assigned['Borough'], new_neigh], axis = 1)
df_replaced.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Whenever we have more than one row per postcode, we will concatenate all neighborhoods into a comma separated list

In [5]:
#group the dataframe by Postcode and Borough and concatenate all neighborhoods into comma separated list
toronto_neighborhoods = df_replaced.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x: ', '.join(x)).to_frame()
toronto_neighborhoods.reset_index(inplace = True)
toronto_neighborhoods.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
#display the shape of the resulting dataframe
toronto_neighborhoods.shape

(103, 3)

<a id="toc2"></a>
## 2. Assign Geographical Coordinates

The next step is to assign the geographical coordinates for each postcode. Instructions on the Coursera submission page suggest to use the Geocoder Python package. Unfortunately I was not able to get usable results from this package. As suggested on the submission page, I decided to use the dataset <a href='http://cocl.us/Geospatial_data'>http://cocl.us/Geospatial_data</a> with predefined coordinates per postal code.  

There is an alternate way by which we could get coordinates and that would be to use the <a href='https://geopy.readthedocs.io/en/stable/'>GeoPy</a> library with the Nominatim geolocator service (instead of Google Maps). However, this library does not return coordinates based on postal codes but rather on neighborhood names. This would mean that we would have to restructure the above dataframe back to what it was before we concatenated neighborhoods into comma separated values. I decided not to go this way, although I suspect it would have been a viable approach to solving this exercise.

In [7]:
#read provided dataset
coords = pd.read_csv('http://cocl.us/Geospatial_data')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We will merge this dataset with our *toronto_neighborhoods* dataset from above.

In [8]:
toronto_neighborhoods = pd.merge(toronto_neighborhoods, coords, left_on = 'Postcode', right_on = 'Postal Code')[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
toronto_neighborhoods.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [9]:
toronto_neighborhoods.shape

(103, 5)